In [1]:
import pandas as pd
import numpy as np
import json
import requests
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
raw = pd.read_csv('weekly_housing_market_data_most_recent.tsv', sep='\t')
raw.head()

,region_id,region_type_id,region_name,region_type,period_begin,period_end,duration,total_homes_sold,total_homes_sold_yoy,average_homes_sold,...,average_adjustment_average_homes_sold,adjusted_average_homes_sold,average_adjustment_average_new_listings,adjusted_average_new_listings,average_adjustment_pending_sales,adjusted_pending_sales,adjusted_average_homes_delisted_yoy,adjusted_average_homes_sold_yoy,adjusted_average_new_listings_yoy,adjusted_pending_sales_yoy
0,2982,5,"Isle of Wight County, VA",county,2019-04-22,2019-04-28,1 weeks,15.0,-0.210526,15.0,...,NaN,15.0,NaN,22.0,NaN,14.0,NaN,-0.210526,0.375,0.555556
1,27160,-2,"Jackson, OH metro area",metro,2020-07-20,2020-07-26,1 weeks,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1462,5,"Jackson County, MN",county,2021-05-10,2021-05-16,1 weeks,1.0,-0.500000,1.0,...,NaN,1.0,NaN,NaN,NaN,2.0,NaN,-0.500000,NaN,0.000000
3,35580,-2,"New Ulm, MN metro area",metro,2020-08-03,2020-08-09,1 weeks,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,-0.500,NaN
4,607,5,"Morgan County, GA",county,2020-08-17,2020-08-23,1 weeks,3.0,0.500000,3.0,...,NaN,3.0,NaN,2.0,NaN,NaN,NaN,0.500000,0.000,NaN


In [9]:
redfin_clean = raw[raw["region_name"].str.contains('Mecklenburg County, NC')]
redfin_clean

,region_id,region_type_id,region_name,region_type,period_begin,period_end,duration,total_homes_sold,total_homes_sold_yoy,average_homes_sold,...,average_adjustment_average_homes_sold,adjusted_average_homes_sold,average_adjustment_average_new_listings,adjusted_average_new_listings,average_adjustment_pending_sales,adjusted_pending_sales,adjusted_average_homes_delisted_yoy,adjusted_average_homes_sold_yoy,adjusted_average_new_listings_yoy,adjusted_pending_sales_yoy
2100,2066,5,"Mecklenburg County, NC",county,2018-10-22,2018-10-28,1 weeks,289.0,-0.306954,289.0,...,NaN,289.0,NaN,365.0,NaN,NaN,NaN,-0.306954,0.051873,NaN
5703,2066,5,"Mecklenburg County, NC",county,2021-04-19,2021-04-25,1 weeks,369.0,0.085294,369.0,...,NaN,369.0,NaN,425.0,NaN,NaN,NaN,0.085294,0.268657,NaN
6571,2066,5,"Mecklenburg County, NC",county,2021-09-20,2021-09-26,1 weeks,417.0,-0.018824,417.0,...,NaN,417.0,NaN,308.0,NaN,NaN,NaN,-0.018824,-0.370143,NaN
8231,2066,5,"Mecklenburg County, NC",county,2018-05-28,2018-06-03,1 weeks,592.0,0.129771,592.0,...,NaN,592.0,NaN,473.0,NaN,NaN,NaN,0.129771,-0.097328,NaN
9949,2066,5,"Mecklenburg County, NC",county,2021-06-28,2021-07-04,1 weeks,635.0,0.081772,635.0,...,NaN,635.0,NaN,360.0,NaN,NaN,NaN,0.081772,-0.172414,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538619,2066,5,"Mecklenburg County, NC",county,2017-04-03,2017-06-25,12 weeks,5484.0,0.059915,457.0,...,NaN,457.0,NaN,549.0,NaN,NaN,NaN,0.060325,0.009191,NaN
1538621,2066,5,"Mecklenburg County, NC",county,2019-07-22,2019-10-13,12 weeks,5422.0,0.115867,451.0,...,NaN,451.0,NaN,470.0,NaN,NaN,NaN,0.116337,0.087963,NaN
1538629,2066,5,"Mecklenburg County, NC",county,2019-04-15,2019-07-07,12 weeks,5587.0,-0.018274,465.0,...,NaN,465.0,NaN,529.0,NaN,NaN,NaN,-0.018987,0.000000,NaN
1538634,2066,5,"Mecklenburg County, NC",county,2017-12-11,2018-03-04,12 weeks,3663.0,-0.028382,305.0,...,NaN,305.0,NaN,361.0,NaN,NaN,NaN,-0.028662,-0.047493,NaN
